# New Age

Fakulteta za matematiko in fiziko - Univerza v Ljubljani

Ivan Slapničar, Sveučilište u Splitu, FESB

April, 2017

## Messages

Easy -> hard?

Present: __age of search__ -> _mathematics_

Future: __age of recommendation__  -> _mathematics_

(BigData, new technologies)


## Easy -> hard

weight:  1 _kg_  -> 1 _kg_ $\pm$ 0.000000001 _kg_

running: 100 _m_ -> 42,195 _m_ ili 100 _m_ < 10 _sek_

mathematics: exam -> state competition  -> [Olympiad](http://www.imo-official.org/)

search, recommending: good -> excellent

## Age of Search

google (and others)


* [50 billion pages](http://www.worldwidewebsize.com/), [3.5 billion querries daily](http://www.internetlivestats.com/google-search-statistics/)
* __PageRank__
* history, context - cookies, storing data (about you), [200+ parameters](http://backlinko.com/google-ranking-factors)

## PageRank

* Graph theory and linear algebra
* [C, Moler, Google PageRank][Mol11]


[Mol11]: https://www.mathworks.com/moler/exm/chapters/pagerank.pdf "C, Moler, 'Google PageRank', mathWorks, 2011."


In [1]:
i = vec([ 2 6 3 4 4 5 6 1 1])
j = vec([ 1 1 2 2 3 3 3 4 6])
G=sparse(i,j,1.0)

6×6 sparse matrix with 9 Float64 nonzero entries:
	[2, 1]  =  1.0
	[6, 1]  =  1.0
	[3, 2]  =  1.0
	[4, 2]  =  1.0
	[4, 3]  =  1.0
	[5, 3]  =  1.0
	[6, 3]  =  1.0
	[1, 4]  =  1.0
	[1, 6]  =  1.0

In [2]:
full(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0

On the graph we do a _random walk_ - probabilty to follow any link is equal:

In [3]:
c=sum(G,1)
n=size(G,1)
for j=1:n
    if c[j]>0
        G[:,j]=G[:,j]/c[j]
    end
end
full(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0       1.0  0.0  1.0
 0.5  0.0  0.0       0.0  0.0  0.0
 0.0  0.5  0.0       0.0  0.0  0.0
 0.0  0.5  0.333333  0.0  0.0  0.0
 0.0  0.0  0.333333  0.0  0.0  0.0
 0.5  0.0  0.333333  0.0  0.0  0.0

* $p$: probability to follow link
* $1-p$: probability to visit some other page (at random)
* google uses $p=0.85$ ?

In [4]:
p=0.85
z = ((1-p)*(c.!=0) + (c.==0))/n
A=p*G+ones(n)*z

6×6 Array{Float64,2}:
 0.025  0.025  0.025     0.875  0.166667  0.875
 0.45   0.025  0.025     0.025  0.166667  0.025
 0.025  0.45   0.025     0.025  0.166667  0.025
 0.025  0.45   0.308333  0.025  0.166667  0.025
 0.025  0.025  0.308333  0.025  0.166667  0.025
 0.45   0.025  0.308333  0.025  0.166667  0.025

## Idea

Let us start a random walk from the vector $x_0=\begin{bmatrix} 1/n \\ 1/n \\ \vdots \\ 1/n \end{bmatrix}$.

The subsequent vectors are  

$$
x_1=A\cdot x_0 \\
x_2=A\cdot x_1 \\
x_3=A\cdot x_2\\
\vdots
$$

When the vector __stabilizes__:

$$
A\cdot x\approx x,
$$

then $x[i]$ is the __rank of the page__ $i$.

In [5]:
function myPageRank(G::SparseMatrixCSC{Float64,Int64},steps::Int)
    p=0.85
    c=sum(G,1)/p
    n=size(G,1)
    for i=1:n
        G.nzval[G.colptr[i]:G.colptr[i+1]-1]./=c[i]
    end
    e=ones(n)
    x=e/n
    z = vec(((1-p)*(c.!=0) + (c.==0))/n)
    for j=1:steps
        x=G*x+(z⋅x)
    end
    x/norm(x,1)
end

myPageRank (generic function with 1 method)

In [6]:
myPageRank(G,15)

6-element Array{Float64,1}:
 0.321024 
 0.170538 
 0.106596 
 0.136795 
 0.0643103
 0.200737 

In [7]:
fieldnames(G)

5-element Array{Symbol,1}:
 :m     
 :n     
 :colptr
 :rowval
 :nzval 

In [8]:
G

6×6 sparse matrix with 9 Float64 nonzero entries:
	[2, 1]  =  0.425
	[6, 1]  =  0.425
	[3, 2]  =  0.425
	[4, 2]  =  0.425
	[4, 3]  =  0.283333
	[5, 3]  =  0.283333
	[6, 3]  =  0.283333
	[1, 4]  =  0.85
	[1, 6]  =  0.85

In [9]:
G.colptr

7-element Array{Int64,1}:
  1
  3
  5
  8
  9
  9
 10

### [Stanford web graph](http://snap.stanford.edu/data/web-Stanford.html)

This is somewhat larger test problem.

In [10]:
W=readdlm("web-Stanford.txt",Int)

2312497×2 Array{Int64,2}:
      1    6548
      1   15409
   6548   57031
  15409   13102
      2   17794
      2   25202
      2   53625
      2   54582
      2   64930
      2   73764
      2   84477
      2   98628
      2  100193
      ⋮        
 281849  165189
 281849  177014
 281849  226290
 281849  243180
 281849  244195
 281849  247252
 281849  281568
 281865  186750
 281865  225872
 281888  114388
 281888  192969
 281888  233184

In [11]:
S=sparse(W[:,2],W[:,1],1.0)

281903×281903 sparse matrix with 2312497 Float64 nonzero entries:
	[6548  ,      1]  =  1.0
	[15409 ,      1]  =  1.0
	[17794 ,      2]  =  1.0
	[25202 ,      2]  =  1.0
	[53625 ,      2]  =  1.0
	[54582 ,      2]  =  1.0
	[64930 ,      2]  =  1.0
	[73764 ,      2]  =  1.0
	[84477 ,      2]  =  1.0
	[98628 ,      2]  =  1.0
	⋮
	[168703, 281902]  =  1.0
	[180771, 281902]  =  1.0
	[266504, 281902]  =  1.0
	[275189, 281902]  =  1.0
	[44103 , 281903]  =  1.0
	[56088 , 281903]  =  1.0
	[90591 , 281903]  =  1.0
	[94440 , 281903]  =  1.0
	[216688, 281903]  =  1.0
	[256539, 281903]  =  1.0
	[260899, 281903]  =  1.0

In [12]:
@time x100=myPageRank(S,100);

  1.771919 seconds (846.68 k allocations: 542.596 MB, 14.09% gc time)


In [13]:
x101=myPageRank(S,101)
maximum(abs,(x100-x101)./x100)

2.349138102570129e-7

In [14]:
sortperm(x100,rev=true), sort(x100,rev=true)

([89073,226411,241454,262860,134832,234704,136821,68889,105607,69358  …  281647,281700,281715,281728,281778,281785,281813,281849,281865,281888],[0.0113029,0.00926783,0.00829727,0.00302312,0.00300128,0.00257173,0.00245371,0.00243079,0.00239105,0.00236401  …  5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7])

## Age of Recommendation

NetFlix, Amazon Prime, PickBox, ... - on-line streaming of movies and shows

[NetFlix](https://www.netflix.com/hr/)

 * [80 million users](https://www.statista.com/statistics/250934/quarterly-number-of-netflix-streaming-subscribers-worldwide/), 5,000 movies
 * [NetFlix Prize](http://www.kdd.org/kdd2014/tutorials/KDD%20-%20The%20Recommender%20Problem%20Revisited.pdf)
 

## Mathematics

Netflix Recommendation Engine is based on approximation of a (large and sparse) matrix
```
M = Users x Movies 
```
using (approximation of) singular value decomposition (SVD): 

* [IncrementalSVD.jl](https://github.com/aaw/IncrementalSVD.jl)
* [A parallel recommendation engine in Julia](http://juliacomputing.com/blog/2016/04/22/a-parallel-recommendation-engine-in-julia.html)

## Similarities

Similarity of users $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[i,:],M[k,:])}{\|M[i,:]\| \cdot \|M[k,:]\|}
$$
Similarity of movies $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[:,i],M[:,k)}{\|M[:,i]\| \cdot \|M[:,k]\|}
$$

## Search

Row $M[u,:]$ - what user $u$ thinks about movies

Column $M[:,m]$ - what users think about movie $m$

Element $M[u,m]$ - what user $u$ thinks about movie $m$.

## Problem

Matrix $M$ is sparse so we do not have enough information. For example, 

```
900188 marks / ( 6040 users x 3706 movies ) = 4%
```

## Approximation

SVD decomposition $M=U\Sigma V^T$ is [approximated by a low-rank matrix](https://en.wikipedia.org/wiki/Low-rank_approximation) (for example. $rank=25$)

![SVD decomposition](svd.png)

The approximation matrix is __full__ and __gives enough good information__.

Prize for efficient approximation algorithm was $\$$ 1.000.000.  

In [15]:
# Pkg.clone("git://github.com/aaw/IncrementalSVD.jl.git")

In [16]:
using IncrementalSVD

In [17]:
whos(IncrementalSVD)

                IncrementalSVD     56 KB     Module
                        Rating    136 bytes  DataType
                     RatingSet    148 bytes  DataType
                  RatingsModel    160 bytes  DataType
             cosine_similarity      0 bytes  IncrementalSVD.#cosine_similarity
          get_predicted_rating      0 bytes  IncrementalSVD.#get_predicted_rati…
                 item_features      0 bytes  IncrementalSVD.#item_features
                   item_search      0 bytes  IncrementalSVD.#item_search
                         items      0 bytes  IncrementalSVD.#items
    load_book_crossing_dataset      0 bytes  IncrementalSVD.#load_book_crossing…
  load_large_movielens_dataset      0 bytes  IncrementalSVD.#load_large_moviele…
  load_small_movielens_dataset      0 bytes  IncrementalSVD.#load_small_moviele…
                          rmse      0 bytes  IncrementalSVD.#rmse
                 similar_items      0 bytes  IncrementalSVD.#similar_items
                 similar_us

In [18]:
rating_set = load_small_movielens_dataset();

Reusing existing downloaded files...


Loading ratings 100% Time: 0:00:04


In [19]:
fieldnames(rating_set)

4-element Array{Symbol,1}:
 :training_set 
 :test_set     
 :user_to_index
 :item_to_index

In [20]:
# The format is (user, movie, mark)
rating_set.training_set

900188-element Array{IncrementalSVD.Rating,1}:
 IncrementalSVD.Rating(5699,251,4.0) 
 IncrementalSVD.Rating(351,217,5.0)  
 IncrementalSVD.Rating(1676,880,3.0) 
 IncrementalSVD.Rating(4860,337,5.0) 
 IncrementalSVD.Rating(5567,794,1.0) 
 IncrementalSVD.Rating(5453,741,4.0) 
 IncrementalSVD.Rating(2529,309,3.0) 
 IncrementalSVD.Rating(2934,643,3.0) 
 IncrementalSVD.Rating(5996,210,5.0) 
 IncrementalSVD.Rating(300,194,3.0)  
 IncrementalSVD.Rating(547,125,5.0)  
 IncrementalSVD.Rating(4482,1349,4.0)
 IncrementalSVD.Rating(2812,1042,4.0)
 ⋮                                   
 IncrementalSVD.Rating(3965,1586,2.0)
 IncrementalSVD.Rating(2624,607,4.0) 
 IncrementalSVD.Rating(3175,384,5.0) 
 IncrementalSVD.Rating(984,1839,4.0) 
 IncrementalSVD.Rating(3769,757,5.0) 
 IncrementalSVD.Rating(4898,362,4.0) 
 IncrementalSVD.Rating(377,663,4.0)  
 IncrementalSVD.Rating(3378,375,5.0) 
 IncrementalSVD.Rating(2279,1273,4.0)
 IncrementalSVD.Rating(2913,959,5.0) 
 IncrementalSVD.Rating(3153,172,4.0) 
 In

In [21]:
rating_set.test_set

100021-element Array{IncrementalSVD.Rating,1}:
 IncrementalSVD.Rating(1676,1119,5.0)
 IncrementalSVD.Rating(5504,273,4.0) 
 IncrementalSVD.Rating(3693,367,3.0) 
 IncrementalSVD.Rating(262,210,2.0)  
 IncrementalSVD.Rating(4821,65,3.0)  
 IncrementalSVD.Rating(1501,438,4.0) 
 IncrementalSVD.Rating(1501,1087,4.0)
 IncrementalSVD.Rating(5831,1186,5.0)
 IncrementalSVD.Rating(3824,144,3.0) 
 IncrementalSVD.Rating(2157,373,4.0) 
 IncrementalSVD.Rating(1501,939,2.0) 
 IncrementalSVD.Rating(4732,214,5.0) 
 IncrementalSVD.Rating(1010,763,2.0) 
 ⋮                                   
 IncrementalSVD.Rating(4022,377,4.0) 
 IncrementalSVD.Rating(3561,393,4.0) 
 IncrementalSVD.Rating(1215,323,4.0) 
 IncrementalSVD.Rating(850,1622,4.0) 
 IncrementalSVD.Rating(2073,563,4.0) 
 IncrementalSVD.Rating(2540,19,4.0)  
 IncrementalSVD.Rating(439,766,5.0)  
 IncrementalSVD.Rating(307,961,5.0)  
 IncrementalSVD.Rating(5963,118,3.0) 
 IncrementalSVD.Rating(3630,922,3.0) 
 IncrementalSVD.Rating(2605,91,3.0)  
 In

In [22]:
# Users and their IDs
rating_set.user_to_index

Dict{AbstractString,Int32} with 6040 entries:
  "4304" => 4304
  "3935" => 3935
  "5422" => 5422
  "5734" => 5734
  "2243" => 2243
  "1881" => 1881
  "5425" => 5425
  "4209" => 4209
  "1907" => 1907
  "2923" => 2923
  "599"  => 599
  "2491" => 2491
  "5944" => 5944
  "228"  => 228
  "2590" => 2590
  "3697" => 3697
  "5031" => 5031
  "2579" => 2579
  "5551" => 5551
  "1880" => 1880
  "2562" => 2562
  "3215" => 3215
  "3991" => 3991
  "4652" => 4652
  "4088" => 4088
  ⋮      => ⋮

In [23]:
# Movies and their IDs
rating_set.item_to_index

Dict{AbstractString,Int32} with 3706 entries:
  "Fried Green Tomatoes (1991)"                                       => 594
  "Milk Money (1994)"                                                 => 1361
  "From Russia with Love (1963)"                                      => 729
  "House II: The Second Story (1987)"                                 => 1247
  "Held Up (2000)"                                                    => 3549
  "Missing in Action 2: The Beginning (1985)"                         => 2177
  "Murder, My Sweet (1944)"                                           => 996
  "Hidden, The (1987)"                                                => 981
  "Cable Guy, The (1996)"                                             => 669
  "Big Kahuna, The (2000)"                                            => 893
  "Addams Family Values (1993)"                                       => 1857
  "Farinelli: il castrato (1994)"                                     => 1945
  "Education of Little T

In [24]:
# We can extract the titles ...
keys(rating_set.item_to_index)

Base.KeyIterator for a Dict{AbstractString,Int32} with 3706 entries. Keys:
  "Fried Green Tomatoes (1991)"
  "Milk Money (1994)"
  "From Russia with Love (1963)"
  "House II: The Second Story (1987)"
  "Held Up (2000)"
  "Missing in Action 2: The Beginning (1985)"
  "Murder, My Sweet (1944)"
  "Hidden, The (1987)"
  "Cable Guy, The (1996)"
  "Big Kahuna, The (2000)"
  "Addams Family Values (1993)"
  "Farinelli: il castrato (1994)"
  "Education of Little Tree, The (1997)"
  "In God's Hands (1998)"
  "Last Man Standing (1996)"
  "Sixth Sense, The (1999)"
  "Star Maps (1997)"
  "Girl, Interrupted (1999)"
  "Stand by Me (1986)"
  "Rob Roy (1995)"
  "Caligula (1980)"
  "Flirting With Disaster (1996)"
  "Hook (1991)"
  "Institute Benjamenta, or This Dream People Call Human Life (1995)"
  ⋮

In [30]:
# or codes
values(rating_set.item_to_index)

Base.ValueIterator for a Dict{AbstractString,Int32} with 3706 entries. Values:
  594
  1361
  729
  1247
  3549
  2177
  996
  981
  669
  893
  1857
  1945
  2759
  2814
  1840
  39
  2670
  32
  85
  449
  2532
  1290
  657
  3397
  ⋮

In [31]:
# Which movies did the user "3000" grade?
user_ratings(rating_set, "3000")

94-element Array{Tuple{SubString{String},Float32},1}:
 ("Defending Your Life (1991)",5.0)                   
 ("Rock, The (1996)",5.0)                             
 ("Gattaca (1997)",5.0)                               
 ("Caddyshack (1980)",5.0)                            
 ("Babe (1995)",5.0)                                  
 ("Brazil (1985)",5.0)                                
 ("Princess Bride, The (1987)",5.0)                   
 ("Time Bandits (1981)",5.0)                          
 ("Twelve Monkeys (1995)",5.0)                        
 ("Dances with Wolves (1990)",5.0)                    
 ("American Beauty (1999)",5.0)                       
 ("One Flew Over the Cuckoo's Nest (1975)",5.0)       
 ("Groundhog Day (1993)",5.0)                         
 ⋮                                                    
 ("Close Encounters of the Third Kind (1977)",2.0)    
 ("Empire Records (1995)",2.0)                        
 ("Star Trek VI: The Undiscovered Country (1991)",1.0)
 ("Mission:

In [32]:
# Let us find the exact title and code for "Blade Runner"
for k in keys(rating_set.item_to_index)
    if contains(k,"Blade")
        println(k)
    end
end

Sling Blade (1996)
Blade (1998)
Blade Runner (1982)
Some Folks Call It a Sling Blade (1993)


LoadError: UnicodeError: invalid character index

In [33]:
get(rating_set.item_to_index,"Blade Runner (1982)",0)

744

In [34]:
# Did the user "3000" grade "Blade Runner" ?
for k in user_ratings(rating_set,"3000")
    if contains(k[1],"Blade")
        println(k)
    end
end

("Blade Runner (1982)",4.0f0)


In [36]:
# Did the user "3000" grade "Citizen Kane" ?
for k in user_ratings(rating_set,"3000")
    if contains(k[1],"Citizen")
        println(k)
    end
end

In [37]:
# This takes about 2.5 minutes
model = train(rating_set, 25);

Computing truncated rank 25 SVD 100% Time: 0:02:29


In [38]:
fieldnames(model)

5-element Array{Symbol,1}:
 :user_to_index
 :item_to_index
 :U            
 :S            
 :V            

In [39]:
model.U

6040×25 Array{Float32,2}:
 0.0109823   0.0120226    0.0166219    …  0.012272     0.0161562  
 0.0142433   0.0111837    0.00921491      0.0118675    0.00920172 
 0.0103782   0.00892511   0.0164168       0.00388684   0.0203382  
 0.00666488  0.00662359   0.0154748       0.0117752    0.0105415  
 0.0137611   0.0162703    0.00156027      0.0182806    0.00993406 
 0.0112359   0.0164399    0.0150594    …  0.00966849   0.0177239  
 0.00896405  0.00741591   0.0184874       0.0133862    0.0123649  
 0.0157949   0.0145479    0.0103715       0.0127705    0.0139319  
 0.0135885   0.00823491   0.0104761       0.00707882   0.013857   
 0.0184266   0.0133142   -0.0110956       0.0266503    0.00839892 
 0.0129194   0.0113756    0.00945753   …  0.00880921   0.0105299  
 0.00632839  0.00840891   0.0146721       0.016305     0.0128159  
 0.0123776   0.00987827   0.0130334       0.0125947    0.0137204  
 ⋮                                     ⋱                          
 0.00693266  0.0133071    0.0141933 

In [40]:
model.S

25-element Array{Float32,1}:
 8194.01  
 2146.71  
  831.351 
 1145.3   
  904.371 
 1125.24  
  685.678 
  381.742 
  353.622 
  259.694 
  240.064 
  310.632 
  301.139 
  226.581 
  254.445 
  282.744 
  276.8   
  187.593 
  176.431 
  137.884 
  156.873 
  136.781 
  189.49  
  138.627 
   93.0921

In [41]:
model.V

3706×25 Array{Float32,2}:
 0.0359893   0.00788367  0.0689725   …   0.00715309   -0.0158565 
 0.0250646   0.0142212   0.0197849      -0.0083699     0.0102397 
 0.0304439   0.0138211   0.0310121       0.00606642   -0.00140682
 0.0318989   0.0120328   0.0543681       0.0141739    -0.0235338 
 0.0316053   0.00760603  0.0551512       0.0193134    -0.0114745 
 0.03464     0.00462663  0.069051    …   0.00661752   -0.00558856
 0.0306795   0.00821352  0.0288004      -0.00375692   -0.00206655
 0.0332597   0.00415005  0.0527167       0.0261144    -0.00797532
 0.0285577   0.00758896  0.0234996       0.00798394    0.00259972
 0.0333331   0.00483039  0.0534006       0.0169534    -0.0115203 
 0.0294347   0.00526755  0.0376264   …  -0.00427749   -0.00189873
 0.0170301   0.0267278   0.00866781      0.00133653    0.0155258 
 0.0260396   0.0184281   0.0135392       0.000288478   0.00221939
 ⋮                                   ⋱                           
 0.00118412  0.00258967  0.00405879      0.0120072

In [42]:
similar_items(model, "Friday the 13th (1980)")

10-element Array{SubString{String},1}:
 "Friday the 13th (1980)"                               
 "Friday the 13th Part 2 (1981)"                        
 "Amityville Horror, The (1979)"                        
 "Omen, The (1976)"                                     
 "Porky's (1981)"                                       
 "Halloween II (1981)"                                  
 "Children of the Corn (1984)"                          
 "Nightmare on Elm Street 4: The Dream Master, A (1988)"
 "Cujo (1983)"                                          
 "Jaws 2 (1978)"                                        

In [43]:
@which similar_items(model, "Friday the 13th (1980)")

similar_items(model::IncrementalSVD.RatingsModel, item) at C:\Users\Ivan\.julia\v0.5\IncrementalSVD\src\util.jl:113

In [44]:
similar_items(model, "Citizen Kane (1941)")

10-element Array{SubString{String},1}:
 "Citizen Kane (1941)"                                                        
 "M*A*S*H (1970)"                                                             
 "Chinatown (1974)"                                                           
 "Rear Window (1954)"                                                         
 "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)"
 "Casablanca (1942)"                                                          
 "African Queen, The (1951)"                                                  
 "Vertigo (1958)"                                                             
 "Lawrence of Arabia (1962)"                                                  
 "Maltese Falcon, The (1941)"                                                 

In [45]:
similar_users(model,"3000")

10-element Array{SubString{String},1}:
 "3000"
 "5572"
 "6030"
 "4353"
 "2874"
 "4252"
 "5619"
 "3106"
 "39"  
 "5613"

In [47]:
# What is the opinion of user "3000" about "Blade Runner" 
# in the approximate model (true mark is 4.0) ?
get_predicted_rating(model, "3000", "Blade Runner (1982)")

4.2369895f0

In [48]:
# What is the opinion of user "3000" about Citizen Kane 
# (no true mark!) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Citizen Kane (1941)")

4.1848273f0

## Thank you for you attention

### Questions?